In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd
url_imdb= "https://assets-datascientest.s3.eu-west-1.amazonaws.com/IMDB_en.html"
page_imdb= urlopen(url_imdb)
bs_imdb = bs(page_imdb, "html.parser")
print("\n".join(bs_imdb.prettify().splitlines()[0:30]))

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;c&&c!==k&&c!==m&&c!==n&&c!==p||a.ec++;c&&c!=k||a.ec

In [8]:
urlbuzz=bs_imdb.findAll("td",class_='titleColumn')
print(urlbuzz[:5])

[<td class="titleColumn">
<a href="/title/tt11564570/?pf_rd_m=A2FGELUUNOQJNL&amp;pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&amp;pf_rd_r=BQWZRBFAM81S7K6ZBPJP&amp;pf_rd_s=center-1&amp;pf_rd_t=15506&amp;pf_rd_i=moviemeter&amp;ref_=chtmvm_tt_1" title="Rian Johnson (dir.), Daniel Craig, Edward Norton">Glass Onion</a>
<span class="secondaryInfo">(2022)</span>
<div class="velocity">1
<span class="secondaryInfo">(
<span class="global-sprite titlemeter up"></span>
1)</span>
</div>
</td>, <td class="titleColumn">
<a href="/title/tt1630029/?pf_rd_m=A2FGELUUNOQJNL&amp;pf_rd_p=ea4e08e1-c8a3-47b5-ac3a-75026647c16e&amp;pf_rd_r=BQWZRBFAM81S7K6ZBPJP&amp;pf_rd_s=center-1&amp;pf_rd_t=15506&amp;pf_rd_i=moviemeter&amp;ref_=chtmvm_tt_2" title="James Cameron (dir.), Sam Worthington, Zoe Saldana">Avatar</a>
<span class="secondaryInfo">(2022)</span>
<div class="velocity">2
<span class="secondaryInfo">(
<span class="global-sprite titlemeter down"></span>
1)</span>
</div>
</td>, <td class="titleColumn">
<a hre

In [22]:
first_film=urlbuzz[0]
print(first_film.find("a").string.strip())

Glass Onion


In [18]:
# gt the ranking of the movie
rank=first_film.find("div",class_="velocity").contents[0]
print(rank)

1



In [17]:
year= first_film.find("span", class_="secondaryInfo").string.strip("()")
print(year)

2022


In [29]:
# rating of the first movie
rating= bs_imdb.findAll("td",class_="ratingColumn imdbRating")
rating_first=rating[0].find('strong').string.strip()
print(rating_first)
print(len(rating))
print(rating[:5])

7,3
100
[<td class="ratingColumn imdbRating">
<strong title="7,3 based on 207 962 user ratings">7,3</strong>
</td>, <td class="ratingColumn imdbRating">
<strong title="7,9 based on 183 394 user ratings">7,9</strong>
</td>, <td class="ratingColumn imdbRating">
<strong title="7,9 based on 664 235 user ratings">7,9</strong>
</td>, <td class="ratingColumn imdbRating">
<strong title="7,4 based on 7 995 user ratings">7,4</strong>
</td>, <td class="ratingColumn imdbRating">
<strong title="7,9 based on 1 289 668 user ratings">7,9</strong>
</td>]


In [42]:
import re
ratings=[]
for rate in rating:
    rating_string=str(rating)
#     print(rating_string)
    rating_value = re.search(r'[0-9]\,[0-9]', rating_string)
#     print(rating_value)
    if rating_value:
        rating_value=rating_value.group()
    else:
        rating_value=None
    ratings.append(rating_value)

In [48]:
# importing in pandas dataframe
import pandas as pd
titles=[]
years=[]
for film in urlbuzz:
    titles.append(film.find("a").string.strip())
    years.append(film.find("span",class_="secondaryInfo").string.strip())
df_imdb=pd.DataFrame(({'Titles':titles,"Year":years,'Rating':ratings}))
df_imdb.head(10)

,Titles,Year,Rating
0,Glass Onion,(2022),"7,3"
1,Avatar,(2022),"7,3"
2,À couteaux tirés,(2019),"7,3"
3,Babylon,(2022),"7,3"
4,Avatar,(2009),"7,3"
5,Les Banshees d'Inisherin,(2022),"7,3"
6,Christmas Story,(1983),"7,3"
7,"Matilda, la comédie musicale",(2022),"7,3"
8,Violent Night,(2022),"7,3"
9,Top Gun: Maverick,(2022),"7,3"


In [50]:
# econf website
import requests
from bs4 import BeautifulSoup as bs
url_meta = "https://www.metacritic.com/browse/movies/score/metascore/year/filtered"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"}
res = requests.get(url_meta, headers=headers).text
soup=bs(res,'html.parser')
soup.prettify().splitlines()[:30]

['<!DOCTYPE html>',
 '<html xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">',
 ' <head>',
 '  <title>',
 '   Best Movies for 2023 - Metacritic',
 '  </title>',
 '  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>',
 '  <meta content="See how well critics are rating the Best Movies for 2023" name="description"/>',
 '  <meta content="Metacritic" name="application-name"/>',
 '  <meta content="#000000" name="msapplication-TileColor"/>',
 '  <meta content="/images/win8tile/76bf1426-2886-4b87-ae1c-06424b6bb8a2.png" name="msapplication-TileImage"/>',
 '  <meta content="618k3mbeki8tar7u6wvrum5lxs5cka" name="facebook-domain-verification">',
 '   <meta content="Best Movies for 2023" property="og:title"/>',
 '   <meta content="website" property="og:type"/>',
 '   <meta content="https://www.metacritic.com/browse/movies/score/metascore/year/filtered" property="og:url"/>',
 '   <meta content="https://static.metacritic.com/images/icons/mc_fb_og.png

In [64]:
films=soup.findAll("td",class_='clamp-summary-wrap')
print(films)

[<td class="clamp-summary-wrap">
<input class="clamp-summary-expand" id="557100" type="checkbox">
<div class="clamp-score-wrap">
<a class="metascore_anchor" href="/movie/past-lives/critic-reviews">
<div class="metascore_w large movie positive">94</div>
</a>
</div>
<span class="title numbered">
                                                                    1.
                                                            </span>
<a class="title" href="/movie/past-lives"><h3>Past Lives</h3></a>
<div class="clamp-details">
<span>June 2, 2023</span>
<span class="cert_rating PG-13"> | PG-13</span>
</div>
<div class="summary">
                        Nora (Greta Lee) and Hae Sung (Teo Yoo), two deeply connected childhood friends, are wrest apart after Nora’s family emigrates from South Korea. Two decades later, they are reunited in New York for one fateful week as they confront notions of destiny and love, and the choices that make a life.
                        </div>
<div class="browse-

In [63]:
# get the title of the first movie
title=films[0].find("h3").string.strip()
print(title)

Past Lives


In [79]:
# get the rating of the first movie
rating=films[0].find("div",class_='metascore_w').string.strip()
print(rating)

94


In [87]:
# get complete lis of movies with ratings
ratings=[]
titles=[]
for film in films:
    title=film.find("h3").get_text(strip=True)
    rating=film.find("div",class_="metascore_w").string.strip()
    titles.append(title)
    ratings.append(rating)
df_meta=pd.DataFrame({"title":titles,"Rating":ratings})
df_meta.head()

,title,Rating
0,Past Lives,94
1,De Humani Corporis Fabrica,92
2,Saint Omer,91
3,Killers of the Flower Moon,89
4,Anatomy of a Fall,89
